In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

In [1]:
batch_size = 100
n_itr = 50000
T = 10
eps = 1e-7
lr = 1e-3
N = 128
M = 20
controller_size = 100
shift_stride = 1
num_shift = 2*shift_stride + 1

In [ ]:
memory = tf.Variable(tf.fill([N, M], eps))
# for feed forward controller
#controller_W = tf.Variable(tf.zeros([N, controller_size]))
#controller_b = tf.Variable(tf.zeros([controller_size]))

k_W = tf.Variable(tf.zeros([controller_size, M]))
k_b = tf.Variable(tf.zeros([M]))
beta_W = tf.Variable(tf.zeros([controller_size, 1]))
beta_b = tf.Variable(tf.zeros([1]))
g_W = tf.Variable(tf.zeros([controller_size, 1]))
g_b = tf.Variable(tf.zeros([1]))
s_W = tf.Variable(tf.zeros([controller_size, num_shift]))
s_b = tf.Variable(tf.zeros([num_shift]))
gamma_W = tf.Variable(tf.zeros([controller_size, 1]))
gamma_b = tf.Variable(tf.zeros([1]))

a_W = tf.Variable(tf.zeros([controller_size, M]))
a_b = tf.Variable(tf.zeros([M]))
e_W = tf.Variable(tf.zeros([controller_size, M]))
e_b = tf.Variable(tf.zeros([M]))

controller = tf.nn.rnn_cell.LSTMCell(controller_size)


In [ ]:
def read(w, mem): #w_t => Nx1
    r = tf.matmul(w, mem)
    return r

def write(w, h, mem_prev): #w_t => Nx1 , e_t => 1xM, a_t = 1xM
    e = tf.nn.relu(tf.matmul(h, e_W) + e_b)
    a = tf.nn.relu(tf.matmul(h, a_W) + a_b)
    
    erase_factor = 1 - tf.multiply(w, e)
    memory_cap = tf.multiply(mem_prev, erase_factor)
    add_factor = tf.multiply(w, a)
    memory_next = memory_cap + add_factor
    assign_op_memory = mem_prev.assign(memory_next)
    return assign_op_memory 

def content_addressing(k, beta, mem_prev): # k => 1xM
    norm_k = tf.nn.l2_normalize(k, dim=1)
    norm_memory = tf.nn.l2_normalize(mem_prev, dim=1)
    cosine_similarity = tf.matmul(norm_memory, norm_k)
    
    w_c = tf.nn.softmax(tf.multiply(beta, cosine_similarity))
    return w_c

def interpolation(w_prev, w_c, g):
    w_g = tf.multiply(w_prev, g) + tf.multiply(w_c, 1 - g)
    return w_g

def convolution_shift(g_w, s):
    for i in xrange(N):
        for j in xrange(num_shift):
            idx = shift_stride + i - j + 1
            if idx < 0: idx = idx + N
            if idx >= N: idx = idx - N
            w = tf.gather(g_w, int(idx)) * tf.gather(s, j)
            output = tf.scatter_add(output, [i], tf.reshape(w, [1, -1]))

def sharpening(w_cap, gamma):
    w = tf.pow(w_cap, gamma)
    w = w / tf.maximum(tf.reduce_sum(w, 2, keep_dims=True), eps)
    return w

def addressing(h, w_prev, mem_prev):
    '''
    The key vector, kt, and key strength, βt, are used to perform 
    content-based addressing of the memory matrix, Mt. 
    The resulting content-based weighting is interpolated with 
    the weighting from the previous time step based on the value 
    of the interpolation gate, gt. The shift weighting, st, 
    determines whether and by how much the weighting is rotated. 
    Finally, depending on γt, the weighting is sharpened 
    and used for memory access.
    
    k => key vector  # 1xM
    beta => key strength # 1
    g => interpolaton gate # 1, (0, 1)
    s => location shift weight # num_shiftx1
    gamma => sharpening wt # 1, >= 1
    '''
    k = tf.nn.relu(tf.matmul(h, k_W) + k_b)
    beta = tf.nn.relu(tf.matmul(h, beta_W) + beta_b)
    g = tf.sigmoid(tf.matmul(h, g_W) + g_b)
    s = tf.nn.softmax(tf.matmul(h, s_W) + s_b)
    gamma = 1 + tf.nn.relu(tf.matmul(h, gamma_W) + gamma_b)
    
    w_c = content_addressing(k, beta, mem_prev)
    w_g = interpolation(w_prev, w_c, g)
    w_cap = convolution_shift(w_g, s)
    w = sharpening(w_cap, gamma)
    return w

In [ ]:
lstm_state = controller.zero_state(batch_size, tf.float32) 

h0 = tf.zeros([batch_size, controller_size])
h = h0
w = w0
for t in range(T):
    r = read(w, memory)
    # input to hidden
    controller_in = tf.concat(1,[x, r, h])
    with tf.variable_scope("controller", reuse=(t != 0)):
        h, controller_state = rnn_enc(controller_in, lstm_state)
    w = addressing(h, w, memory)
    
    write(w, e, a, memory)